In [207]:
import numpy as np
from astropy import constants as const
from astropy import units as u
from astropy.table import QTable
import pandas as pd
import emcee
from matplotlib import pyplot as plt

from cluster import c, Cluster, temp_from_vdisp

In [24]:
# consts
n=0 # only collisions for now
norm = 1/4 # normalizing factor for accretion rate
mu = 1  # mean molecular weight of gas, 1 for proton gas (hydrogen)
m_chi = 1e-3*u.GeV

In [56]:
# load in a dataset here
galwcls=pd.read_csv('data/galwcls.dat', sep='|', header=None)
cls_data = {'sig500': galwcls[:][8],
            'M500': galwcls[:][11],
            'r200': galwcls[:][13],
            'sig200':galwcls[:][15],
            'err_neg':galwcls[:][16],
            'err_pos':galwcls[:][17],
            'M200':galwcls[:][18]}
units = {'sig500': u.km/u.s,
            'M500': u.Msun,
            'r200': u.Mpc,
            'sig200': u.km/u.s,
            'err_neg':u.km/u.s,
            'err_pos':u.km/u.s,
            'M200': u.Msun, }
cls_table = QTable(cls_data[], units=units)

In [18]:
i = 1
test_cluster = Cluster(
    cls_table['r200'][i], 
    cls_table['M200'][i], 
    cls_table['sig200'][i], 
    m500=cls_table['M500'][i])

In [133]:
# for 1 free parameter, Tx=0 approximation (small mx)
def T_b(sigma0, cluster):
    V=cluster.volume.to(u.cm**3)
    x = (3*const.c*c(n)*V*cluster.rho_dm*cluster.rho_b*sigma0/(cluster.m_b+m_chi)**2).to(1/u.s)
    leading_factors = (norm * 4*np.pi *const.c ** -3).to(u.s**3/u.cm**3)
    gm2 = ((const.G * cluster.bh_mass()) ** 2).to(u.cm**6/u.s**4)
    frac = ((mu * cluster.m_b) ** (5 / 2) / cluster.adiabatic_idx ** (3 / 2)).to(u.GeV**(5/2))
    nb = (2 * cluster.n_e).to(u.cm ** (-3)) # baryon number density
    D = (cluster.epsilon*leading_factors*gm2*frac*(1/nb**(2/3))**(-3/2)) # removed k_B from original function because we are working in GeV here
    return (((D*np.sqrt(cluster.m_b))/x)**(1/3)).to(u.GeV, equivalencies=u.temperature_energy())

In [115]:
T_b(1e-15*u.cm**2, test_cluster).to(u.GeV, equivalencies=u.temperature_energy()) # predicted cluster temp

<Quantity 2.29240968e-07 GeV>

In [53]:
test_cluster.baryon_temp # actual cluster temp (from data)

<Quantity 8.96643311e-06 GeV>

In [54]:
def variance(err_neg, err_pos):
    n_temp = temp_from_vdisp(err_neg)
    p_temp = temp_from_vdisp(err_pos)
    return n_temp + p_temp

In [97]:
def chi_squared(T_model, T_data, variance): # take lists of model prediction, data, and variance of same length
    chi_squared_sum = 0
    for i in range(len(T_model)):
        chi_squared_sum+=(T_model[i]-T_data[i])**2/variance[i]**2
    return chi_squared_sum
        

In [225]:
T_data = [temp_from_vdisp(v) for v in cls_table['sig200']]
clusters = [Cluster(cls_table['r200'][i], cls_table['M200'][i], cls_table['sig200'][i], m500=cls_table['M500'][i]) for i in range(galwcls.shape[0])]
T_model = [T_b(1e-10*u.cm**2, c) for c in clusters]
var = variance(cls_table['err_neg'], cls_table['err_pos'])

In [226]:
X2 = chi_squared(T_model, T_data, var)

In [227]:
def log_likelihood(p0, T_data, var):
    if p0<0:
        return -np.inf
    T_model = [T_b(p0, c) for c in clusters]
    X2 = chi_squared(T_model, T_data, var)
    return (-X2/2)[0]

In [228]:
#chi_squared(T_model, T_data, var)
log_likelihood(sigma0[0], T_data, var)

<Quantity -45461.79217825>

In [229]:
ndim, nwalkers = 1, 5
# initialize random sigma0s
sigma0 = np.power(10, np.random.uniform(low=-30, high=-10, size=(nwalkers, ndim)))*u.cm**2
sigma0

<Quantity [[2.81086544e-26],
           [3.77926548e-20],
           [5.10554443e-24],
           [2.42372063e-23],
           [6.72819718e-12]] cm2>

In [230]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood, args=[T_data, var])

In [231]:
state = sampler.run_mcmc(sigma0, 100, progress=True)
sampler.reset()

In [ ]:
sampler.run_mcmc(state, 10000, progress=True)

In [ ]:
def plot_loghist(x, bins):
    logbins = np.logspace(np.log10(np.min(x)),np.log10(np.max(x)),bins+1)
    plt.hist(x, bins=logbins)
    plt.xscale('log')
    plt.yscale('log')

In [ ]:
samples = sampler.get_chain(flat=True)
plot_loghist(samples[:, 0], 100)
plt.xlabel(r"$\sigma_0$")
plt.ylabel(r"$p(\sigma_0)$")



In [ ]:
np.median(samples)

In [ ]:
print(
    "Mean acceptance fraction: {0:.3f}".format(
        np.mean(sampler.acceptance_fraction)
    )
)

In [ ]:
print(
    "Mean autocorrelation time: {0:.3f} steps".format(
        np.mean(sampler.get_autocorr_time())
    )
)